In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import codecs

from constrained_decoding import create_constrained_decoder
from constrained_decoding.translation_model.nematus_tm import NematusTranslationModel

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX TITAN X (0000:03:00.0)


In [3]:
def load_config(filename):
    # defaults -- params which are inputs to `nematus/translate.py`, but aren't part of the persisted *.json config
    translate_config = {
        "return_alignment": False
    }
    config = json.loads(codecs.open(filename, encoding='utf8').read())
    return dict(translate_config, **config)

In [4]:
# Working: test with hard-coded config and model paths

configs = [
    '/media/1tb_drive/nematus_ape_experiments/amunmt_ape_pretrained/system/models/src-pe/model.npz.json'
]

models = [
    '/media/1tb_drive/nematus_ape_experiments/amunmt_ape_pretrained/system/models/src-pe/model.4-best.averaged.npz'
]

In [5]:
# remember Nematus needs _encoded_ utf8
configs = [load_config(f) for f in configs]

# build ensembled TM
nematus_tm = NematusTranslationModel(models, configs, model_weights=None)


Building f_init... Done
Building f_next.. Done


In [11]:
model = nematus_tm

In [12]:
# from constrained_decoding.server import run_imt_server

In [13]:
decoder = create_constrained_decoder(nematus_tm)
#     app.decoders = {k: create_constrained_decoder(v) for k, v in models.items()}

In [14]:
# model_dict = {('en', 'de'): nematus_tm}

# run_imt_server(models=model_dict)

In [19]:
#     model = app.models[(source_lang, target_lang)]
    # TODO: create constrained decoders from all models
#     decoder = app.decoders[(source_lang, target_lang)]
    # WORKING: call the correct model
    # WORKING: remember we support multiple inputs for each model (i.e. each model may be an ensemble where sub-models
    # accept different inputs)

    # Note: for now we only support one source input

length_factor = 1.5
beam_size = 5
n_best = 5

    
source_sentence = u'Help me .'
inputs = [source_sentence]

mapped_inputs = model.map_inputs(inputs)
print(mapped_inputs)

input_constraints = []
# if constraints is not None:
#     input_constraints = model.map_constraints(constraints)

# import ipdb; ipdb.set_trace()
start_hyp = model.start_hypothesis(mapped_inputs, input_constraints)

search_grid = decoder.search(start_hyp=start_hyp, constraints=input_constraints,
                             max_hyp_len=int(round(len(mapped_inputs[0][0]) * length_factor)),
                             beam_size=beam_size)

best_output, best_alignments = decoder.best_n(search_grid, model.eos_token, n_best=n_best,
                                              return_model_scores=False, return_alignments=True,
                                              length_normalization=True)

if n_best > 1:
    # start from idx 1 to cut off `None` at the beginning of the sequence
    # separate each n-best list with newline
    decoder_output = u'\n'.join([u' '.join(s[0][1:]) for s in best_output]) + u'\n\n'
else:
    # start from idx 1 to cut off `None` at the beginning of the sequence
    decoder_output = u' '.join(best_output[0][1:])

# Note alignments are always an n-best list (may be n=1)
# if write_alignments is not None:
#     with codecs.open(write_alignments, 'a+', encoding='utf8') as align_out:
#         align_out.write(json.dumps([a.tolist() for a in best_alignments]) + u'\n')


[array([[[1526],
        [ 732],
        [   3],
        [   0]]])]
[[[1526]
  [ 732]
  [   3]
  [   0]]]


In [20]:
decoder_output

u'Help me .\nHilfe me .\nHilfe .\nHilfe bei mir .\nHilfe mir .\n\n'